# 01 - Tool Testing Notebook

**Purpose:** This notebook tests each individual tool in the AI Travel Planner system to ensure they work correctly before integration.

**Tools Tested:**
- Flight Search (SerpAPI)
- Hotel Search (SerpAPI)
- Weather Forecast (OpenWeatherMap)
- Place Search (Google Places & Tavily)
- Currency Conversion
- Expense Calculator

**Expected Outcome:** All tools should return valid data without errors.

## Setup and Imports

In [ ]:
import os
import sys
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add src to path (adjust based on your project structure)
sys.path.append('../src')

print("✅ Environment loaded")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. Test Flight Search Tool

In [ ]:
from src.tools.flight_serpapi_tool import search_flights

# Test Parameters
test_origin = "Mumbai"
test_destination = "Delhi"
test_date = (datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d")

print(f"🔍 Testing Flight Search: {test_origin} → {test_destination} on {test_date}")
print("="*60)

# Execute search
result = search_flights.invoke({
    "origin": test_origin,
    "destination": test_destination,
    "travel_date": test_date
})

# Parse and display results
flight_data = json.loads(result)

if "error" in flight_data:
    print(f"❌ Error: {flight_data['error']}")
else:
    print(f"✅ Found {flight_data['count']} flights")
    print(f"\n📊 Sample Flights:")
    
    for i, flight in enumerate(flight_data['flights'][:3], 1):
        print(f"\n{i}. {flight['Airline']} {flight['FlightNumber']}")
        print(f"   Price: {flight['PriceFormatted']}")
        print(f"   Route: {flight['Route']}")
        print(f"   Duration: {flight['Duration']}")
        print(f"   Category: {flight['Category']}")

## 2. Test Hotel Search Tool

In [ ]:
from src.tools.hotel_serpapi_tool import search_hotels

# Test Parameters
test_location = "Delhi"
checkin = (datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d")
checkout = (datetime.now() + timedelta(days=10)).strftime("%Y-%m-%d")

print(f"🔍 Testing Hotel Search: {test_location}")
print(f"📅 Check-in: {checkin}, Check-out: {checkout}")
print("="*60)

# Execute search
result = search_hotels.invoke({
    "location": test_location,
    "check_in_date": checkin,
    "check_out_date": checkout
})

# Parse and display results
hotel_data = json.loads(result)

if "error" in hotel_data:
    print(f"❌ Error: {hotel_data['error']}")
else:
    stats = hotel_data['stats']
    print(f"✅ Found {len(hotel_data['hotels'])} hotels")
    print(f"   Budget: {stats['budget']}")
    print(f"   Moderate: {stats['moderate']}")
    print(f"   Luxury: {stats['luxury']}")
    
    print(f"\n📊 Sample Hotels by Category:")
    
    for category in ['Budget', 'Moderate', 'Luxury']:
        cat_hotels = [h for h in hotel_data['hotels'] if h['Cat'] == category]
        if cat_hotels:
            hotel = cat_hotels[0]
            print(f"\n{category}:")
            print(f"   {hotel['Name']}")
            print(f"   Rating: {hotel['Rat']} ⭐")
            print(f"   Price: ₹{hotel['Price']:,}/night (Total: ₹{hotel['Total']:,})")
            print(f"   Location: {hotel['Loc'][:50]}...")

## 3. Test Weather Forecast Tool

In [ ]:
from src.tools.weather_info_tool import get_weather_forecast

# Test Parameters
test_city = "Delhi"
test_travel_date = (datetime.now() + timedelta(days=3)).strftime("%Y-%m-%d")

print(f"🔍 Testing Weather Forecast: {test_city}")
print(f"📅 Travel Date: {test_travel_date}")
print("="*60)

# Execute weather lookup
result = get_weather_forecast.invoke({
    "city": test_city,
    "travel_date": test_travel_date
})

print(result)

if "Error" in result or "error" in result:
    print("\n❌ Weather API error detected")
else:
    print("\n✅ Weather data retrieved successfully")

## 4. Test Place Search Tools

In [ ]:
from src.tools.place_search_tool import PlaceSearchTool

# Initialize place search tools
place_tools = PlaceSearchTool()
tools_list = place_tools.place_search_tool_list

# Get individual tools
search_attractions = tools_list[0]
search_restaurants = tools_list[1]
search_activities = tools_list[2]

test_place = "Paris"

print(f"🔍 Testing Place Search Tools for: {test_place}")
print("="*60)

### 4.1 Test Attractions Search

In [ ]:
print("\n🎨 Testing Attractions Search...")
attractions_result = search_attractions.invoke({"place": test_place})
print(attractions_result[:500] + "..." if len(attractions_result) > 500 else attractions_result)

### 4.2 Test Restaurant Search

In [ ]:
print("\n🍽️ Testing Restaurant Search...")
restaurants_result = search_restaurants.invoke({"place": test_place})
print(restaurants_result[:500] + "..." if len(restaurants_result) > 500 else restaurants_result)

### 4.3 Test Activities Search

In [ ]:
print("\n🎯 Testing Activities Search...")
activities_result = search_activities.invoke({"place": test_place})
print(activities_result[:500] + "..." if len(activities_result) > 500 else activities_result)

## 5. Test Currency Conversion Tool

In [ ]:
from src.tools.currency_conversion_tool import CurrencyConverterTool

# Initialize currency converter
currency_tool = CurrencyConverterTool()
convert_currency = currency_tool.currency_converter_tool_list[0]

print("🔍 Testing Currency Conversion")
print("="*60)

# Test conversion
test_amount = 1000
from_currency = "USD"
to_currency = "INR"

try:
    result = convert_currency.invoke({
        "amount": test_amount,
        "from_currency": from_currency,
        "to_currency": to_currency
    })
    
    print(f"✅ Conversion successful")
    print(f"   {test_amount} {from_currency} = {result:.2f} {to_currency}")
    print(f"   Exchange Rate: 1 {from_currency} = {result/test_amount:.4f} {to_currency}")
except Exception as e:
    print(f"❌ Error: {e}")

## 6. Test Expense Calculator Tool

In [ ]:
from src.tools.expense_calculator_tool import CalculatorTool

# Initialize calculator
calc_tool = CalculatorTool()
tools = calc_tool.calculator_tool_list

estimate_hotel_cost = tools[0]
calculate_total = tools[1]
calculate_daily_budget = tools[2]

print("🔍 Testing Expense Calculator")
print("="*60)

### 6.1 Test Hotel Cost Calculation

In [ ]:
price_per_night = 5000.0
nights = 3

hotel_total = estimate_hotel_cost.invoke({
    "price_per_night": price_per_night,
    "total_days": nights
})

print(f"\n✅ Hotel Cost Calculation:")
print(f"   ₹{price_per_night}/night × {nights} nights = ₹{hotel_total}")

### 6.2 Test Total Expense Calculation

In [ ]:
flight_cost = 8000.0
food_cost = 6000.0
activities_cost = 4000.0

total = calculate_total.invoke({
    "costs": [flight_cost, hotel_total, food_cost, activities_cost]
})

print(f"\n✅ Total Expense Calculation:")
print(f"   Flights: ₹{flight_cost}")
print(f"   Hotels: ₹{hotel_total}")
print(f"   Food: ₹{food_cost}")
print(f"   Activities: ₹{activities_cost}")
print(f"   TOTAL: ₹{total}")

### 6.3 Test Daily Budget Calculation

In [ ]:
trip_days = 5

daily_budget = calculate_daily_budget.invoke({
    "total_cost": total,
    "days": trip_days
})

print(f"\n✅ Daily Budget Calculation:")
print(f"   Total: ₹{total} ÷ {trip_days} days = ₹{daily_budget:.2f}/day")

## Summary

This notebook tested all individual tools:

1. ✅ **Flight Search** - SerpAPI integration working
2. ✅ **Hotel Search** - SerpAPI hotel search functioning
3. ✅ **Weather Forecast** - OpenWeatherMap API tested
4. ✅ **Place Search** - Google Places & Tavily fallback tested
5. ✅ **Currency Conversion** - Exchange rate API working
6. ✅ **Expense Calculator** - Mathematical operations verified

All tools are ready for integration into the multi-agent workflow.